In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [2]:
df = pd.read_csv("cardio_train.csv", delimiter=";", index_col=0)
df.head()

pcs = MinMaxScaler()
variaveis_continuas = ["age", "height", "weight", "ap_hi", "ap_lo", "cholesterol"]
df[variaveis_continuas] = pcs.fit(df[variaveis_continuas]).transform(df[variaveis_continuas])

df.gender = df.gender.apply(lambda genero: 0 if genero == 2 else genero)

x, y = df.iloc[:,:-1].to_numpy(), df.iloc[:,-1].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)
X_train = X_train.T
X_test = X_test.T
y_train = y_train.reshape((1, X_train.shape[1]))
y_test = y_test.reshape((1, X_test.shape[1]))

### Cada layer terá uma matriz de pesos W e um vetor bias b associado. A dimensão da matriz W é definida com a quantidade de linhas sendo igual a quantidade de nodos no layer e a quantidade colunas como a quantidade de inputs do layer. O vetor bias terá um bias para cada nodo layer

In [3]:
def get_parametros_iniciais(tamanho_layers):
    np.random.seed(1)
    parametros = []
    for indice in range(1,len(tamanho_layers)):
        parametros.append([np.random.randn(tamanho_layers[indice], tamanho_layers[indice-1]) * (2/np.sqrt(tamanho_layers[indice-1])),
                           np.zeros((tamanho_layers[indice], 1))])
    return parametros
        

### Definição das funções de ativação utilizadas: sigmóide e relu

In [4]:
def sigmoid(Z):
    """
    Implements the sigmoid activation in numpy
    
    Arguments:
    Z -- numpy array of any shape
    
    Returns:
    A -- output of sigmoid(z), same shape as Z
    cache -- returns Z as well, useful during backpropagation
    """
    
    A = 1/(1+np.exp(-Z))
    
    return A

def softsign(Z):
    A = np.divide(Z, (1+np.abs(Z)))
    return A

def relu(Z):
    """
    Implement the RELU function.

    Arguments:
    Z -- Output of the linear layer, of any shape

    Returns:
    A -- Post-activation parameter, of the same shape as Z
    cache -- a python dictionary containing "A" ; stored for computing the backward pass efficiently
    """
    
    A = np.maximum(0,Z)

    return A

### Esquema forward propagation
<img src="imagens/model_architecture_kiank.png" style="width:600px;height:300px;">

In [5]:
def forward_propagation(X, parametros):
    cache = []
    A = X
    quantidade_layers = len(parametros)
    for indice_layer in range(0, quantidade_layers-1): # foward propagation até o ultimo layer antes do layer output
        W = parametros[indice_layer][0]
        b = parametros[indice_layer][1]
        Z = np.dot(W, A) + b
        cache.append((A, Z, W, b))
        A = relu(Z)
    W, b = parametros[indice_layer+1] # como é um problema de classificao, o último layer deve obrigatoriamente ter
                                      #  a funcao sigmoide como funcao de ativação
    
    Z = np.dot(W, A) + b
    cache.append((A, Z, W, b))
    A = sigmoid(Z)
    
    return A, cache

### A função custo usada aqui é definida como: $$-\frac{1}{m} \sum\limits_{i = 1}^{m} (y^{(i)}\log\left(a^{[L] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right)) $$

In [14]:
def get_custo(A, Y, lambd, parametros):    
    m = Y.shape[1]
#     if 0 in A or 1 in a:
#         print("zerro")
    l2 = lambd/(2*m) * np.sum([np.sum(np.square(w)) for w in parametros[:][0]])
    custo = (1./m) * (-np.dot(Y,np.log(A).T) - np.dot(1-Y, np.log(1-A).T)) + l2
    custo = float(np.squeeze(custo))
    return custo

In [7]:
def derivada_relu(dA, Z):
    dZ = np.array(dA, copy=True) 
    dZ[Z <= 0] = 0
    return dZ

def derivada_softsign(dA, Z):
    return np.multiply(dA, np.divide(1, np.multiply(1+Z, 1+Z)))

def derivada_sigmoide(dA, Z):
    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s)    
    return dZ

In [8]:
def backward_propagation(A, cache, Y):
    #print(A.shape)
    m = A.shape[1]
    Y = Y.reshape(A.shape)
    gradientes = []
    
    dA = - (np.divide(Y, A) - np.divide(1 - Y, 1 - A)) # derivada do custo em função de A
    A_prev, Z, W, b = cache[-1]
    dZ = derivada_sigmoide(dA, Z)

    dA_layer_anterior = np.dot(W.T, dZ)
    dW = np.dot(dZ, A_prev.T) / m
    db = np.sum(dZ, axis=1, keepdims=True) / m
    gradientes.append([dW, db])

    for cache_ in cache[::-1][1:]:
        dA = dA_layer_anterior
        A_prev, Z, W, b = cache_
        dZ = derivada_relu(dA, Z)

        dA_layer_anterior = np.dot(W.T, dZ)
        dW = np.dot(dZ, A_prev.T) / m
        db = np.sum(dZ, axis=1, keepdims=True) / m
        gradientes.append([dW, db])
        
    return gradientes[::-1]


In [9]:
def get_parametros_atualizados(parametros, gradientes, m, lambd, taxa_aprendizado=0.05):
    L = len(parametros) 
    for l in range(L):
        parametros[l][0] -= taxa_aprendizado * (gradientes[l][0] + lambd/m * parametros[l][0])
        parametros[l][1] -= taxa_aprendizado * gradientes[l][1] 
    return parametros
    

In [41]:
def fit(X, Y, taxa_aprendizado, iteracoes, tamanhos_layers, lambd=0.7):
    global gradientes
    global parametros
    parametros = get_parametros_iniciais(tamanhos_layers)
    custos = []
    for iteracao in range(iteracoes):
        A, cache = forward_propagation(X, parametros)
        custo = get_custo(A, Y, lambd, parametros)
        custos.append(custo)
        if iteracao % 1000 == 0:
            print("Iteração: {} | Custo: {}".format(iteracao, custo))
        gradientes = backward_propagation(A, cache, Y)
        parametros = get_parametros_atualizados(parametros, gradientes, X.shape[1], lambd, taxa_aprendizado)
    
    return parametros, custos        

In [34]:
def predict(X, Y, parametros):
    A, cache = forward_propagation(X, parametros)
    return np.round(A)

In [12]:
# def converte_array_em_parametros(array, tamanho_layers):
#     parametros = []
#     for index in range(1, len(tamanho_layers)):
#         w_shape_linear = tamanho_layers[index]*tamanho_layers[index-1]
#         w = array[:w_shape_linear].reshape((tamanho_layers[index], tamanho_layers[index-1]))
#         b = array[w_shape_linear:w_shape_linear + tamanho_layers[index]].reshape((tamanho_layers[index], 1))
#         parametros.append([w, b])
#         array = array[w_shape_linear + tamanho_layers[index]:]
#     return parametros

# def verifica_backpropagation(parametros, gradientes, tamanho_layers, X, Y, epsilon=1e-7):
#     parameters_values = np.concatenate([np.concatenate([parametro[0].flatten(), parametro[1].flatten()]) 
#                                        for parametro in parametros])
#     grads =  np.concatenate([np.concatenate([gradiente[0].flatten(), gradiente[1].flatten()]) 
#                                        for gradiente in gradientes])
    
#     num_parameters = parameters_values.shape[0]
#     J_plus = np.zeros((num_parameters, 1))
#     J_minus = np.zeros((num_parameters, 1))
#     gradapprox = np.zeros((num_parameters, 1))
    
#     for i in range(num_parameters):
        
#         # Compute J_plus[i]. Inputs: "parameters_values, epsilon". Output = "J_plus[i]".
#         # "_" is used because the function you have to outputs two parameters but we only care about the first one
#         ### START CODE HERE ### (approx. 3 lines)
#         thetaplus =  np.copy(parameters_values)                                       # Step 1
#         thetaplus[i] = thetaplus[i] + epsilon                                   # Step 2
#         A, _ =  forward_propagation(X, converte_array_em_parametros(thetaplus, tamanho_layers))  # Step 3
#         J_plus[i][0] = get_custo(A, Y, 0.7, converte_array_em_parametros(thetaplus, tamanho_layers))
#         ### END CODE HERE ###
        
#         # Compute J_minus[i]. Inputs: "parameters_values, epsilon". Output = "J_minus[i]".
#         ### START CODE HERE ### (approx. 3 lines)
#         thetaminus = np.copy(parameters_values)                                       # Step 1
#         thetaminus[i] = thetaminus[i] - epsilon                                 # Step 2       
#         A, _ = forward_propagation(X, converte_array_em_parametros(thetaminus, tamanho_layers)) # Step 3
#         J_minus[i][0] = get_custo(A, Y, 0.7, converte_array_em_parametros(thetaminus, tamanho_layers))
#         ### END CODE HERE ###
        
#         # Compute gradapprox[i]
#         ### START CODE HERE ### (approx. 1 line)
#         gradapprox[i] = (J_plus[i] - J_minus[i]) / (2 * epsilon)
#         ### END CODE HERE ###
    
#     # Compare gradapprox to backward propagation gradients by computing difference.
#     ### START CODE HERE ### (approx. 1 line)
#     numerator = np.linalg.norm(grads - gradapprox)                                     # Step 1'
#     denominator = np.linalg.norm(grads) + np.linalg.norm(gradapprox)                   # Step 2'
#     difference = numerator / denominator                                              # Step 3'
#     ### END CODE HERE ###

#     if difference > 1e-7:
#         print("\033[93m" + "There is a mistake in the backward propagation! difference = " + str(difference) + "\033[0m")
#     else:
#         print("\033[92m" + "Your backward propagation works perfectly fine! difference = " + str(difference) + "\033[0m")
    
#     return difference
    

In [ ]:
tamanho_layers = [11, 25, 5, 3, 1]
gradientes = []
parametros = []
parametros, custos = fit(X_train, y_train, 0.05, 50000, tamanho_layers)

Iteração: 0 | Custo: 1.3531599175681845
Iteração: 1000 | Custo: 0.6401567957135242
Iteração: 2000 | Custo: 0.6327863026034765
Iteração: 3000 | Custo: 0.6318209809444724
Iteração: 4000 | Custo: 0.6305476762910055
Iteração: 5000 | Custo: 0.6301878129691637
Iteração: 6000 | Custo: 0.6302138327016902
Iteração: 7000 | Custo: 0.6294049887037406
Iteração: 8000 | Custo: 0.629028232682224
Iteração: 9000 | Custo: 0.6288087594502474
Iteração: 10000 | Custo: 0.6284439292547017
Iteração: 11000 | Custo: 0.6282097184136229
Iteração: 12000 | Custo: 0.6278294947851617
Iteração: 13000 | Custo: 0.6278970331056625
Iteração: 14000 | Custo: 0.6282035039561675
Iteração: 15000 | Custo: 0.6274104317066351
Iteração: 16000 | Custo: 0.6271447926073501
Iteração: 17000 | Custo: 0.6274893735204532


In [49]:
train_pred = predict(X_train, y_train, parametros)

In [50]:
f1_score(y_train[0], train_pred[0])

0.6066019571646917

In [51]:
test_pred = predict(X_test, y_test, parametros)

In [52]:
f1_score(y_test[0], test_pred[0])

0.6005085015096139